## DIA 055: Implementación de un Sistema de Colas de Mensajes con Redis y Flask

Hoy implementaremos un sistema de colas de mensajes con Redis y Flask para manejar tareas en segundo plano.
Esto es útil para procesos largos como:
✅ Procesamiento de imágenes 🖼️
✅ Envío de correos electrónicos 📧
✅ Generación de reportes 📊

Usaremos Redis como sistema de colas y RQ (Redis Queue) para gestionar tareas asíncronas.

🖥️ Código Completo (app.py)
python
Copiar
Editar
import os
import redis
import logging
from flask import Flask, request, jsonify
from rq import Queue
from worker import process_task

# Configuración básica de Flask
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'supersecretkey')

# Configuración de Redis
redis_conn = redis.Redis(host='localhost', port=6379, db=0)
task_queue = Queue(connection=redis_conn)

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Endpoint para Crear una Tarea en la Cola
# ---------------------------
@app.route('/enqueue', methods=['POST'])
def enqueue_task():
    """
    Agrega una tarea a la cola para ser procesada en segundo plano.
    """
    data = request.get_json()
    task_type = data.get("task_type", "default_task")

    job = task_queue.enqueue(process_task, task_type)
    
    return jsonify({"message": "Tarea encolada", "job_id": job.get_id()}), 200

# ---------------------------
# Endpoint para Ver el Estado de una Tarea
# ---------------------------
@app.route('/status/<job_id>', methods=['GET'])
def task_status(job_id):
    """
    Obtiene el estado de una tarea encolada.
    """
    job = task_queue.fetch_job(job_id)
    if job is None:
        return jsonify({"error": "Tarea no encontrada"}), 404

    return jsonify({"job_id": job.get_id(), "status": job.get_status()}), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True, port=5002)
⚙️ Código del Worker (worker.py)
El worker ejecutará las tareas en segundo plano.

python
Copiar
Editar
import time
import logging
from rq import Worker, Queue, Connection
import redis

# Configuración de Redis
redis_conn = redis.Redis(host='localhost', port=6379, db=0)

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Función para Procesar Tareas
# ---------------------------
def process_task(task_type):
    """
    Simula una tarea pesada según el tipo de tarea recibido.
    """
    logger.info(f"🔄 Procesando tarea: {task_type}")
    time.sleep(5)  # Simulación de proceso largo
    logger.info(f"✅ Tarea {task_type} completada")

    return f"Tarea {task_type} completada"

# ---------------------------
# Ejecutar el Worker para Procesar la Cola de Mensajes
# ---------------------------
if __name__ == '__main__':
    with Connection(redis_conn):
        worker = Worker([Queue()])
        worker.work()
🚀 Cómo Probar el Sistema
1️⃣ Iniciar Redis
Si no tienes Redis instalado, usa Docker:

sh
Copiar
Editar
docker run --name redis -p 6379:6379 -d redis
O si lo tienes instalado localmente:

sh
Copiar
Editar
redis-server
2️⃣ Iniciar el Worker
Abre una terminal y ejecuta:

sh
Copiar
Editar
python worker.py
3️⃣ Agregar una Tarea a la Cola
En otra terminal, prueba con cURL:

sh
Copiar
Editar
curl -X POST http://127.0.0.1:5002/enqueue \
     -H "Content-Type: application/json" \
     -d '{"task_type": "procesar_reporte"}'
📩 Respuesta esperada:

json
Copiar
Editar
{
    "message": "Tarea encolada",
    "job_id": "f3a8c7b2-1d5b-48f7-92d2-d6b1bbf8f42b"
}
4️⃣ Consultar el Estado de la Tarea

sh
Copiar
Editar
curl -X GET http://127.0.0.1:5002/status/f3a8c7b2-1d5b-48f7-92d2-d6b1bbf8f42b
📌 Estados posibles:
✅ queued → En espera en la cola.
✅ started → En proceso.
✅ finished → Completada.

🔍 Explicación de las Principales Implementaciones
🔹 📌 Uso de Redis como sistema de colas

Redis almacena las tareas hasta que un worker las procese.
Esto permite que las tareas no bloqueen la API.
🔹 ⚙️ Creación de Workers con RQ (Redis Queue)

El worker escucha la cola y ejecuta las tareas.
Se pueden ejecutar múltiples workers en paralelo para procesar tareas más rápido.
🔹 📨 API con dos Endpoints
✅ /enqueue → Agrega una tarea a la cola.
✅ /status/<job_id> → Consulta el estado de una tarea.

🔹 ⏳ Simulación de Procesos Largos

Se usa time.sleep(5) en process_task() para simular tareas pesadas.